# **Mercedes-Benz Greener Manufacturing**

<img src="https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/2020-mercedes-benz-eqc-drive-102-1558020251.jpg?crop=1.00xw:0.822xh;0,0.122xh&resize=980:*" width="600px">

## Описание

Задачей соревнования является предсказать **время**, которое необходимо прохождения фазы тестирования. Набор данных представляет собой различные перестановки характеристик автомобилей Mercedes-Benz. Уменьшение времени работы алгоритма сможет способствовать также снижению выбросов углекислого газа без снижения стандартов Daimler.

Набор данных содержит анонимизированный набор переменных (пользовательские функции) в автомобиле Mercedes. 
Например, переменная может быть 4WD, это может быть добавленная пневматическая подвеска или головной дисплей.

***y*** - переменная, которую необходимо предсказать, это время (в секундах), которое потребовалось автомобилю, чтобы пройти тестирование для каждой переменной

Переменные, содержащие буквами - ***категориальные***. Переменные с 0/1 имеют ***двоичный*** тип.

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
colors = ['#001c57','#50248f','#a6a6a6','#38d1ff']
sns.palplot(sns.color_palette(colors))

## 1. Основная информация о данных

In [ ]:
train = pd.read_csv('../input/mercedes-benz-greener-manufacturing/train.csv.zip')
test = pd.read_csv('../input/mercedes-benz-greener-manufacturing/test.csv.zip')
sub = pd.read_csv('../input/mercedes-benz-greener-manufacturing/sample_submission.csv.zip')

print("Train shape : ", train.shape)
print("Test shape : ", test.shape)

In [ ]:
train.head()

## 2. Exploratory Data Analysis

### 2.1 Анализ целевой переменной

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
sns.distplot(train.y.values, bins=50, color=colors[1])
plt.title('Распределение целевой переменной - y\n',fontsize=15)
plt.xlabel('Значение в секундах'); plt.ylabel('Кол-во (частота)');

plt.subplot(122)
sns.boxplot(train.y.values, color=colors[3])
plt.title('Распределение целевой переменной - y\n',fontsize=15)
plt.xlabel('Значение в секундах'); 

In [ ]:
train.y.describe()

Целевая переменная имеет стандартное распределение примерно от 72 до 140 секунд.
Первый и третий квартили лежат в диапозоне примерно от 91 до 109 сек, медиана в 100 сек, заметим также что есть выбросы начиная от 140 сек, которые мы может удалить из обучающей выборки, так как эти значения будут добавлять шум в наш алгоритм.

### 2.2 Типы данных

In [ ]:
train.dtypes.value_counts()

In [ ]:
train.dtypes[train.dtypes=='float']

In [ ]:
dtype_df = train.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()

In [ ]:
train.dtypes[train.dtypes=='object']

In [ ]:
obj = train.dtypes[train.dtypes=='object'].index
for i in obj:
    print(i, train[i].unique())

### 2.3 Пропущенные значения

In [ ]:
train.isna().sum()[train.isna().sum()>0]

### 2.4 Категориальные переменные

In [ ]:
fig,ax = plt.subplots(len(obj), figsize=(20,70))

for i, col in enumerate(obj):
    sns.boxplot(x=col, y='y', data=train, ax=ax[i])

Что видим из графиков:

1) Так как есть необходимость в снижении времени тестирования, то наилучшими значениями в переменных, при которых это время минимально az и bc (X0), y (X1), n (X2), x и h (X5) (гипотеза: могут влиять на y?)

2) Переменные X3, X5, X6, X8 имеют похожие распределения значений, где нет особых различий внутри фичи между значениями в разрезе средних и квартилей

3) X0 и X2 имеют наибольшее разнообразие внутри переменных, что может потенциально говорить об большей полезности данных фичей 

### 2.5 Числовые переменные

In [ ]:
num = train.dtypes[train.dtypes=='int'].index[1:]

Мы имеем набор числовых переменных, где значение задано 1, либо 0, поэтому нет необходимости проводить объемный ананлиз. В данном случае нам должно быть интересно, меняется ли значение показателей внутри переменных, для этого исследуем дисперсию данных переменных, используем при этом функцию var(), и выберем только те, где дисперсия нулевая (то есть всегда 0, либо 1 на всем датасете в разрезе переменной)

In [ ]:
nan_num = []
for i in num:
    if (train[i].var()==0):
        print(i, train[i].var())
        nan_num.append(i)

Мы получили несколько таких переменных, из анализа мы можем их удалить, так как они никак не будут влиять на таргет, тем самым мы повышаем происзводительность работы алгоритма

In [ ]:
train = train.drop(columns=nan_num, axis=1)

### 2.6 Корреляционный анализ

Чтобы мы смогли сделать корреляционный анализ и для категориальных переменных, то перед этим нужно преобразовать эти переменные, использовав LabelEncoder(), при преобразовании значений в двойчный вид мы не сможем отследить взаимосвязь той или иной переменной + мы должны учесть тестовый набор, так как его значения будут участвовать при нахождении таргета

In [ ]:
for i in obj:
    le = LabelEncoder()
    le.fit(list(train[i].values) + list(train[i].values))
    train[i] = le.transform(list(train[i].values))

In [ ]:
train[obj].head()

In [ ]:
corr = train[train.columns[1:10]].corr()

fig,ax = plt.subplots(figsize=(7,6))
sns.heatmap(corr, vmax=.7, square=True,annot=True);

Среди категориальных переменных мы не обнаружили прямую зависимость с таргетом y

In [ ]:
threshold = 1

corr_all = train.drop(columns=obj, axis=1).corr()
corr_all.loc[:,:] =  np.tril(corr_all, k=-1) 

In [ ]:
already_in = set()
result = []
for col in corr_all:
    perfect_corr = corr_all[col][corr_all[col] == threshold ].index.tolist()
    if perfect_corr and col not in already_in:
        already_in.update(set(perfect_corr))
        perfect_corr.append(col)
        result.append(perfect_corr)

In [ ]:
result

При анализе числовых переменных, мы обнаружили, что часть из них имееет прямую корреляцую с другими, поэтому во избежании мультиколлинеарности можем удалить переменные корреляцией 1 (одну из группы оставить), либо использовать регуляризацию, чтобы алгоритм сделал это в автоматичексом режиме.

Как еще мы можем удалить такие переменные без корреляции? Все просто, удаляем дубликаты в разрезе столбцов.

In [ ]:
train.T.drop_duplicates().T